In [4]:
from ultralytics import YOLO
model = YOLO("modelFolder/model_v7.pt")
results = model(source="validimage/1_Color.png",device="cpu", project = "results",name="test6d", conf=0.8 )
for result in results:
    image_points = result.masks.xyn
    print(len(image_points))
    


image 1/1 c:\Team_Project-1\validimage\1_Color.png: 384x640 1 box, 936.5ms
Speed: 0.0ms preprocess, 936.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)


1


In [5]:
from crop_ply import *
xyz_output= cropPly(image_points,"validimage/1.ply")
print(xyz_output)

[PointCloud with 1461 points.]


In [4]:
import cv2
import pyrealsense2 as rs
import numpy as np
from ultralytics import YOLO

# RealSense 카메라 스트리밍 설정
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.color, 1280, 720, rs.format.bgr8, 30)
config.enable_stream(rs.stream.depth, 640,360, rs.format.z16,30)
pipeline.start(config)

# YOLO 모델 로드 (예: Darknet 또는 다른 라이브러리 사용)
model = YOLO("modelFolder/model_v7.pt")

# 클래스별 고유한 색상 정의
class_colors = {
     0: (0, 255, 0),   # 클래스 0의 색상 (예: 녹색)
    1: (255, 0, 0),   # 클래스 1의 색상 (예: 빨강)
    2: (0, 0, 255),   # 클래스 2의 색상 (예: 파랑)
    3: (255, 255, 0), # 클래스 3의 색상 (예: 노랑)
    4: (128, 0, 128), # 클래스 4의 색상 (예: 보라)
    
}

while True:
    # RealSense 카메라 프레임 가져오기
    frames = pipeline.wait_for_frames()
    color_frame = frames.get_color_frame()
    if not color_frame:
        continue

    # RealSense 프레임을 OpenCV 이미지로 변환
    color_image = np.asanyarray(color_frame.get_data())

    # YOLO를 사용하여 객체 검출 및 세그멘테이션 수행
    results = model(color_image, conf=0.8,device="cpu")
   # 결과를 color_image에 오버레이
    for result in results:
        print(result.boxes)
       
        for box,cls,conf in zip(result.boxes.xywh.tolist(), result.boxes.cls.tolist(),result.boxes.conf.tolist()):
            # print("r",box)
            # print("cls",cls)
            x, y, w, h = box
            x1, y1, x2, y2 = int(x - w / 2), int(y - h / 2), int(x + w / 2), int(y + h / 2)
            label = f"{model.names[int(cls)]}: {conf:.2f}"
            
            color_image = cv2.rectangle(color_image, (x1, y1), (x2, y2), class_colors[cls], 2)
            color_image = cv2.putText(color_image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2)

    # 결과를 color_image에 오버레이 또는 다른 방식으로 표시

    # 화면에 결과 표시
    cv2.imshow("YOLO Segmentation", color_image)

    # 'q' 키를 눌러 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 정리 및 종료
pipeline.stop()
cv2.destroyAllWindows()



0: 384x640 1 box, 1 energdrink, 884.5ms
Speed: 0.0ms preprocess, 884.5ms inference, 8.0ms postprocess per image at shape (1, 3, 384, 640)
WARNING  'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.



ultralytics.engine.results.Boxes object with attributes:

boxes: tensor([[5.4651e+02, 3.4224e+02, 7.2615e+02, 5.0804e+02, 9.5698e-01, 1.0000e+00],
        [1.1994e+03, 4.5363e+02, 1.2797e+03, 5.7467e+02, 9.0629e-01, 3.0000e+00]])
cls: tensor([1., 3.])
conf: tensor([0.9570, 0.9063])
data: tensor([[5.4651e+02, 3.4224e+02, 7.2615e+02, 5.0804e+02, 9.5698e-01, 1.0000e+00],
        [1.1994e+03, 4.5363e+02, 1.2797e+03, 5.7467e+02, 9.0629e-01, 3.0000e+00]])
id: None
is_track: False
orig_shape: (720, 1280)
shape: torch.Size([2, 6])
xywh: tensor([[ 636.3329,  425.1424,  179.6395,  165.8050],
        [1239.5671,  514.1487,   80.3252,  121.0425]])
xywhn: tensor([[0.4971, 0.5905, 0.1403, 0.2303],
        [0.9684, 0.7141, 0.0628, 0.1681]])
xyxy: tensor([[ 546.5132,  342.2399,  726.1527,  508.0449],
        [1199.4045,  453.6274, 1279.7297,  574.6699]])
xyxyn: tensor([[0.4270, 0.4753, 0.5673, 0.7056],
        [0.9370, 0.6300, 0.9998, 0.7982]])


0: 384x640 1 box, 1 energdrink, 924.0ms
Speed: 2.0ms preprocess, 924.0ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)
WARNING  'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.



ultralytics.engine.results.Boxes object with attributes:

boxes: tensor([[5.4682e+02, 3.3740e+02, 7.2734e+02, 5.0809e+02, 9.6141e-01, 1.0000e+00],
        [1.1961e+03, 4.4766e+02, 1.2794e+03, 5.7464e+02, 9.0016e-01, 3.0000e+00]])
cls: tensor([1., 3.])
conf: tensor([0.9614, 0.9002])
data: tensor([[5.4682e+02, 3.3740e+02, 7.2734e+02, 5.0809e+02, 9.6141e-01, 1.0000e+00],
        [1.1961e+03, 4.4766e+02, 1.2794e+03, 5.7464e+02, 9.0016e-01, 3.0000e+00]])
id: None
is_track: False
orig_shape: (720, 1280)
shape: torch.Size([2, 6])
xywh: tensor([[ 637.0820,  422.7466,  180.5173,  170.6930],
        [1237.7715,  511.1486,   83.2544,  126.9760]])
xywhn: tensor([[0.4977, 0.5871, 0.1410, 0.2371],
        [0.9670, 0.7099, 0.0650, 0.1764]])
xyxy: tensor([[ 546.8234,  337.4001,  727.3407,  508.0931],
        [1196.1443,  447.6606, 1279.3987,  574.6366]])
xyxyn: tensor([[0.4272, 0.4686, 0.5682, 0.7057],
        [0.9345, 0.6218, 0.9995, 0.7981]])


0: 384x640 1 box, 1 energdrink, 1061.6ms
Speed: 0.0ms preprocess, 1061.6ms inference, 10.2ms postprocess per image at shape (1, 3, 384, 640)
WARNING  'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.



...

ultralytics.engine.results.Boxes object with attributes:

boxes: tensor([[5.4680e+02, 3.3729e+02, 7.2708e+02, 5.0848e+02, 9.6252e-01, 1.0000e+00],
        [1.1967e+03, 4.4902e+02, 1.2794e+03, 5.7510e+02, 8.9772e-01, 3.0000e+00]])
cls: tensor([1., 3.])
conf: tensor([0.9625, 0.8977])
data: tensor([[5.4680e+02, 3.3729e+02, 7.2708e+02, 5.0848e+02, 9.6252e-01, 1.0000e+00],
        [1.1967e+03, 4.4902e+02, 1.2794e+03, 5.7510e+02, 8.9772e-01, 3.0000e+00]])
id: None
is_track: False
orig_shape: (720, 1280)
shape: torch.Size([2, 6])
xywh: tensor([[ 636.9406,  422.8857,  180.2816,  171.1841],
        [1238.0879,  512.0579,   82.7080,  126.0821]])
xywhn: tensor([[0.4976, 0.5873, 0.1408, 0.2378],
        [0.9673, 0.7112, 0.0646, 0.1751]])
xyxy: tensor([[ 546.7998,  337.2937,  727.0814,  508.4778],
        [1196.7339,  449.0168, 1279.4419,  575.0989]])
xyxyn: tensor([[0.4272, 0.4685, 0.5680, 0.7062],
        [0.9349, 0.6236, 0.9996, 0.7987]])


0: 384x640 1 box, 1 energdrink, 871.8ms
Speed: 10.1ms preprocess, 871.8ms inference, 8.0ms postprocess per image at shape (1, 3, 384, 640)
WARNING  'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.



ultralytics.engine.results.Boxes object with attributes:

boxes: tensor([[5.4683e+02, 3.3726e+02, 7.2702e+02, 5.0843e+02, 9.6372e-01, 1.0000e+00],
        [1.1963e+03, 4.4809e+02, 1.2794e+03, 5.7467e+02, 8.9796e-01, 3.0000e+00]])
cls: tensor([1., 3.])
conf: tensor([0.9637, 0.8980])
data: tensor([[5.4683e+02, 3.3726e+02, 7.2702e+02, 5.0843e+02, 9.6372e-01, 1.0000e+00],
        [1.1963e+03, 4.4809e+02, 1.2794e+03, 5.7467e+02, 8.9796e-01, 3.0000e+00]])
id: None
is_track: False
orig_shape: (720, 1280)
shape: torch.Size([2, 6])
xywh: tensor([[ 636.9258,  422.8446,  180.1917,  171.1666],
        [1237.8408,  511.3798,   83.1375,  126.5852]])
xywhn: tensor([[0.4976, 0.5873, 0.1408, 0.2377],
        [0.9671, 0.7102, 0.0650, 0.1758]])
xyxy: tensor([[ 546.8300,  337.2614,  727.0216,  508.4279],
        [1196.2721,  448.0872, 1279.4095,  574.6724]])
xyxyn: tensor([[0.4272, 0.4684, 0.5680, 0.7061],
        [0.9346, 0.6223, 0.9995, 0.7982]])


0: 384x640 1 box, 1 energdrink, 1048.3ms
Speed: 2.0ms preprocess, 1048.3ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
WARNING  'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.


ultralytics.engine.results.Boxes object with attributes:

boxes: tensor([[5.4678e+02, 3.3708e+02, 7.2647e+02, 5.0855e+02, 9.6438e-01, 1.0000e+00],
        [1.1965e+03, 4.4832e+02, 1.2794e+03, 5.7501e+02, 8.9226e-01, 3.0000e+00]])
cls: tensor([1., 3.])
conf: tensor([0.9644, 0.8923])
data: tensor([[5.4678e+02, 3.3708e+02, 7.2647e+02, 5.0855e+02, 9.6438e-01, 1.0000e+00],
        [1.1965e+03, 4.4832e+02, 1.2794e+03, 5.7501e+02, 8.9226e-01, 3.0000e+00]])
id: None
is_track: False
orig_shape: (720, 1280)
shape: torch.Size([2, 6])
xywh: tensor([[ 636.6234,  422.8131,  179.6903,  171.4654],
        [1237.9589,  511.6634,   82.9277,  126.6904]])
xywhn: tensor([[0.4974, 0.5872, 0.1404, 0.2381],
        [0.9672, 0.7106, 0.0648, 0.1760]])
xyxy: tensor([[ 546.7782,  337.0804,  726.4685,  508.5458],
        [1196.4950,  448.3182, 1279.4227,  575.0086]])
xyxyn: tensor([[0.4272, 0.4682, 0.5676, 0.7063],
        [0.9348, 0.6227, 0.9995, 0.7986]])


In [6]:
import open3d as o3d
import numpy as np

# 포인트 클라우드를 시각화
for cloud in xyz_output:
    # 포인트 클라우드의 경계 상자 계산
    aabb = cloud.get_axis_aligned_bounding_box()
    aabb.color = (1, 0, 0)

    obb = cloud.get_oriented_bounding_box()
    obb.color = (0, 1, 0)

    # 클라우드 포인트의 중심 위치 계산
    center = np.mean(np.asarray(cloud.points), axis=0)
    print(center)

    # 중앙 위치에 좌표 프레임 메시 생성
    mesh = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.07)  # size를 조절하여 좌표 프레임 크기 설정 가능

    # 중앙 위치에 좌표 프레임 메시를 이동
    mesh.translate(center)

    # 시각화
    o3d.visualization.draw_geometries([cloud, aabb, obb, mesh])

    # PCA를 수행하여 주요 주성분 벡터를 찾습니다.
    points = np.asarray(cloud.points)
    covariance_matrix = np.cov(points, rowvar=False)
    eigenvalues, eigenvectors = np.linalg.eigh(covariance_matrix)

    # 주요 주성분 벡터와 각각의 축과의 각도를 계산합니다.
    x_axis = np.array([1, 0, 0])  # x 축
    y_axis = np.array([0, 1, 0])  # y 축
    z_axis = np.array([0, 0, 1])  # z 축

    angle_x_rad = np.arccos(np.dot(eigenvectors[:, -1], x_axis) / (np.linalg.norm(eigenvectors[:, -1]) * np.linalg.norm(x_axis)))
    angle_x_deg = np.degrees(angle_x_rad)

    angle_y_rad = np.arccos(np.dot(eigenvectors[:, -1], y_axis) / (np.linalg.norm(eigenvectors[:, -1]) * np.linalg.norm(y_axis)))
    angle_y_deg = np.degrees(angle_y_rad)

    angle_z_rad = np.arccos(np.dot(eigenvectors[:, -1], z_axis) / (np.linalg.norm(eigenvectors[:, -1]) * np.linalg.norm(z_axis)))
    angle_z_deg = np.degrees(angle_z_rad)

    
    print(f"(x 축 기준): {angle_x_deg}도")
    print(f"(y 축 기준): {angle_y_deg}도")
    print(f"(z 축 기준): {angle_z_deg}도")
    
    o3d.io.write_point_cloud("robot/test.ply", cloud)


[  -0.040005    -0.02612    -0.64369]
(x 축 기준): 177.53971780029164도
(y 축 기준): 90.01150427849596도
(z 축 기준): 92.46025526947014도


In [3]:
import open3d as o3d
import numpy as np

# 포인트 클라우드를 시각화
for cloud in xyz_output:
    # 포인트 클라우드의 경계 상자 계산
    aabb = cloud.get_axis_aligned_bounding_box()
    aabb.color = (1, 0, 0)

    obb = cloud.get_oriented_bounding_box()
    obb.color = (0, 1, 0)

    # 클라우드 포인트의 중심 위치 계산
    center = np.mean(np.asarray(cloud.points), axis=0)
    print(center)

    # 중앙 위치에 좌표 프레임 메시 생성
    mesh = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.07)  # size를 조절하여 좌표 프레임 크기 설정 가능

    # 중앙 위치에 좌표 프레임 메시를 이동
    mesh.translate(center)

    # 시각화
    o3d.visualization.draw_geometries([cloud, aabb, obb, mesh])

    # PCA를 수행하여 주요 주성분 벡터를 찾습니다.
    points = np.asarray(cloud.points)
    covariance_matrix = np.cov(points, rowvar=False)
    eigenvalues, eigenvectors = np.linalg.eigh(covariance_matrix)

     # 주요 주성분 벡터와 중심 위치를 시각화
    lines = []
    for i in range(3):  # 주요 주성분 벡터의 개수는 3개입니다.
        eigenvector = eigenvectors[:, i]  # i번째 주요 주성분 벡터 가져오기

        # 중심 위치에서 주요 주성분 벡터를 시작하도록 끝점 좌표 계산
        endpoint = center + eigenvector * 0.1  # 벡터의 크기를 조절하여 시각화 크기 설정 가능

        # 주요 주성분 벡터를 라인 메시로 생성하여 시각화
        line = o3d.geometry.LineSet()
        line.points = o3d.utility.Vector3dVector(np.array([center, endpoint]))
        line.lines = o3d.utility.Vector2iVector(np.array([[0, 1]]))

        # 주요 주성분 벡터에 따라 다른 색상 설정
        if i == 0:
            line.colors = o3d.utility.Vector3dVector(np.array([[1, 0, 0]]))  # 빨간색
        elif i == 1:
            line.colors = o3d.utility.Vector3dVector(np.array([[0, 1, 0]]))  # 녹색
        elif i == 2:
            line.colors = o3d.utility.Vector3dVector(np.array([[0, 0, 1]]))  # 파란색
        
        lines.append(line)
    

      # 시각화에 추가
    o3d.visualization.draw_geometries([cloud, aabb, obb, mesh, *lines])

    # 포인트 클라우드를 PLY 파일로 저장
    o3d.io.write_point_cloud("robot/test.ply", cloud)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


NameError: name 'xyz_output' is not defined

In [2]:
import pyrealsense2 as rs
import cv2
import numpy as np

# RealSense 카메라 객체 생성
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)

# 카메라 시작
pipeline.start(config)

# RealSense 카메라 메트릭스 및 왜곡 계수 얻기
frames = pipeline.wait_for_frames()
color_frame = frames.get_color_frame()
intrinsics = color_frame.profile.as_video_stream_profile().intrinsics

# 카메라 메트릭스 출력
print("카메라 메트릭스:")
print(f"Focal Length: ({intrinsics.fx}, {intrinsics.fy})")
print(f"Principal Point: ({intrinsics.ppx}, {intrinsics.ppy})")

# 체스보드의 가로 및 세로 내부 코너 개수
board_width = 9
board_height = 6

# 체스보드 코너 좌표 초기화
objp = np.zeros((board_width * board_height, 3), np.float32)
objp[:, :2] = np.mgrid[0:board_width, 0:board_height].T.reshape(-1, 2)

# 3D 포인트 배열과 2D 포인트 배열 초기화
objpoints = []  # 3D 포인트 (실제 3D 체스보드 코너)
imgpoints = []  # 2D 포인트 (이미지 상의 체스보드 코너)

# VideoCapture 객체 생성 (카메라에 연결)
cap = cv2.VideoCapture(1)

image_counter = 0  # 이미지 카운터 초기화

while True:
    ret, frame = cap.read()  # 프레임 캡처

    if not ret:
        continue  # 프레임을 읽지 못하면 다음 반복으로 넘어감

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # 체스보드 코너 탐지
    ret, corners = cv2.findChessboardCorners(gray, (board_width, board_height), None)
    
    if ret:
        objpoints.append(objp)
        imgpoints.append(corners)

        # 체스보드 코너 표시
        cv2.drawChessboardCorners(frame, (board_width, board_height), corners, ret)
        
        # 이미지 저장
        image_name = f"calibration_image_{image_counter}.jpg"
        cv2.imwrite(image_name, frame)
        image_counter += 1
        # 'q' 키를 누르면 루프 종료
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # 이미지 표시
    cv2.imshow("Chessboard", frame)

# 캘리브레이션 수행 (이미지 저장 후에 수행)
if len(objpoints) > 0 and len(imgpoints) > 0:
    ret, camera_matrix, dist_coeffs, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

    # 결과 출력
    print("카메라 메트릭스:")
    print(camera_matrix)
    print("\n왜곡 계수:")
    print(dist_coeffs)
else:
    print("체스보드를 감지하는 데 실패했습니다. 이미지를 확보하거나 체스보드를 올바르게 배치하세요.")

# 카메라 캡처 해제 및 창 닫기
cap.release()
cv2.destroyAllWindows()


카메라 메트릭스:
Focal Length: (614.6367797851562, 614.6595458984375)
Principal Point: (313.7270812988281, 246.65493774414062)


KeyboardInterrupt: 